In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import re

from ames_preprocessing import restrict_col_list, get_compressed_ames, transformed_df
from ames_model_helper import * 

In [2]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV

# Import and clean

In [3]:
data = pd.read_csv('data/Ames_Housing_Price_Data.csv', index_col = 0).drop_duplicates().reset_index(drop = True)

## Manual adjustments

In [4]:
housing = data.copy()

#Do not appear to be legit garages; remove
housing.at[531,'GarageType'] = np.nan
housing.at[531,'GarageCars'] = np.nan
housing.at[531,'GarageArea'] = 0
housing.at[433, 'GarageType'] = np.nan

#Fill using known basement finish type
housing.at[2433, 'BsmtFinType2'] = housing.at[2433, 'BsmtFinType1']

#Fill missing exposure & electrical with most frequently occuring
housing.at[813, 'BsmtExposure'] = 'No'
housing.at[1201, 'BsmtExposure'] = 'No'
housing.at[2441, 'Electrical'] = 'SBrkr'

#Fix remodel year which makes no sense
housing.at[2033, 'YearRemodAdd'] = housing.at[2033, 'YearBuilt']

## Create cleaned/compressed dataset & feature lists

In [5]:
data_dict = get_compressed_ames(housing)

In [6]:
housing = data_dict['housing']
areas = data_dict['areas']
frontage = data_dict['frontage']
miscval = data_dict['miscval']
conditions = data_dict['conditions']
inspect10pt = data_dict['inspect10pt']
inspect5pt = data_dict['inspect5pt']
inspections = data_dict['inspections']
dates = data_dict['dates']
counts = data_dict['counts']
other_cats = data_dict['categoricals']

In [7]:
housing['remod_age'] = housing['YrSold'] - housing['YearRemodAdd']
housing.drop('YearRemodAdd', axis = 1, inplace = True)
dates = restrict_col_list(dates, housing)
dates.append('remod_age')

In [8]:
categoricals = other_cats+conditions+inspections
numeric = [x for x in housing.columns if x not in categoricals+['PID', 'SalePrice']]
assert set(housing[numeric].select_dtypes(include=np.number).columns) == set(numeric)

# Create training and test sets

In [9]:
features = housing.drop(['PID', 'SalePrice'], axis = 1)
target = housing.SalePrice

#Create train/test indices
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = .3, random_state = 11)
train_idx = X_train.index
test_idx = X_test.index

# Encoding

In [10]:
oe = ColumnTransformer([
    ('pass', 'passthrough', numeric),
    
    #pass a list of columns which you want to be OE
    ('oe', OrdinalEncoder(), categoricals)
])

features_encoded = pd.DataFrame(oe.fit_transform(features), columns = numeric + categoricals)
assert (features_encoded[numeric] == features[numeric]).all().all()

X_enc_train = features_encoded.loc[train_idx,:] 
X_enc_test = features_encoded.loc[test_idx,:] 

# Baseline Random Forest Regressor

In [11]:
rfr = RandomForestRegressor()
rfr.fit(X_enc_train, y_train)
print('Train score: %s' %rfr.score(X_enc_train,y_train))
print('Test score: %s' %rfr.score(X_enc_test,y_test))

Train score: 0.9832832658100903
Test score: 0.8849081844048697


Massive overfitting. Tune first.

## Tune

With bootstrapping

In [12]:
param_grid = {'n_estimators': 2 ** np.arange(5,13,2), 'max_depth': 2 ** np.arange(1,7), 'max_features': np.linspace(0,1,6)[1:],
 'bootstrap':[True], 'oob_score':[True], 'n_jobs': [-1], 'random_state':[11]}

In [16]:
gsrfr = RandomizedSearchCV(RandomForestRegressor(), param_distributions = param_grid, n_jobs = -1, return_train_score = True, n_iter = 15)
gsrfr.fit(X_enc_train,y_train)
best_rfr = gsrfr.best_estimator_

/Users/zachstone/miniforge3/envs/env_tf/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [17]:
print(f'The best CV score was {gsrfr.best_score_} using parameters {gsrfr.best_params_}')

The best CV score was 0.881068380669832 using parameters {'random_state': 11, 'oob_score': True, 'n_jobs': -1, 'n_estimators': 2048, 'max_features': 0.4, 'max_depth': 64, 'bootstrap': True}


In [18]:
best_rfr.fit(X_enc_train, y_train)
print('Train score: %s' %best_rfr.score(X_enc_train,y_train))
print('Test score: %s' %best_rfr.score(X_enc_test,y_test))

Train score: 0.9848431220495929
Test score: 0.8883495045472805


In [ ]:
#Very long training time
# gsrfr = GridSearchCV(RandomForestRegressor(), param_grid = param_grid, n_jobs = -1, return_train_score = True)
# gsrfr.fit(X_enc_train,y_train)
# best_rfr = gsrfr.best_estimator_

In [49]:
# print(f'The best CV score was {gsrfr.best_score_} using parameters {gsrfr.best_params_}')

The best CV score was 0.883509373880855 using parameters {'bootstrap': True, 'max_depth': 16, 'max_features': 0.2, 'n_estimators': 512, 'n_jobs': -1, 'oob_score': True, 'random_state': 11}


Grid search results:
The best CV score was 0.883509373880855 using parameters {'bootstrap': True, 'max_depth': 16, 'max_features': 0.2, 'n_estimators': 512, 'n_jobs': -1, 'oob_score': True, 'random_state': 11}

Even after extensive tuning, continues to overfit.""